In [1]:
pip install tensorflow keras nltk 

  Using cached tensorflow-2.8.0-cp39-cp39-win_amd64.whl (438.0 MB)
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4847 sha256=88718904328b6793c6f1d87f883f92600d0765a5d18d2d3cc3645daf046cc383
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\b6\0d\90\0d1bbd99855f99cb2f6c2e5ff96f8023fad8ec367695f7d72d
Successfully built termcolor
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam, SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
import json
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [47]:
intents_file = open('Intent.json').read()
intents = json.loads(intents_file)

In [48]:
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

for intent in intents['intents']:
    for pattern in intent['text']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)        
        #add documents in the corpus
        documents.append((word, intent['intent']))
        # add to our classes list
        if intent['intent'] not in classes:
            classes.append(intent['intent'])
print(documents)
print(words)

[(['Hi'], 'Greeting'), (['Hi', 'there'], 'Greeting'), (['Hola'], 'Greeting'), (['Hello'], 'Greeting'), (['Hello', 'there'], 'Greeting'), (['Hya'], 'Greeting'), (['Hya', 'there'], 'Greeting'), (['My', 'user', 'is', 'Adam'], 'GreetingResponse'), (['This', 'is', 'Adam'], 'GreetingResponse'), (['I', 'am', 'Adam'], 'GreetingResponse'), (['It', 'is', 'Adam'], 'GreetingResponse'), (['My', 'user', 'is', 'Bella'], 'GreetingResponse'), (['This', 'is', 'Bella'], 'GreetingResponse'), (['I', 'am', 'Bella'], 'GreetingResponse'), (['It', 'is', 'Bella'], 'GreetingResponse'), (['How', 'are', 'you', '?'], 'CourtesyGreeting'), (['Hi', 'how', 'are', 'you', '?'], 'CourtesyGreeting'), (['Hello', 'how', 'are', 'you', '?'], 'CourtesyGreeting'), (['Hola', 'how', 'are', 'you', '?'], 'CourtesyGreeting'), (['How', 'are', 'you', 'doing', '?'], 'CourtesyGreeting'), (['Hope', 'you', 'are', 'doing', 'well', '?'], 'CourtesyGreeting'), (['Hello', 'hope', 'you', 'are', 'doing', 'well', '?'], 'CourtesyGreeting'), (['Good

In [49]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

145 documents
22 classes ['Clever', 'CourtesyGoodBye', 'CourtesyGreeting', 'CourtesyGreetingResponse', 'CurrentHumanQuery', 'GoodBye', 'Gossip', 'Greeting', 'GreetingResponse', 'Jokes', 'NameQuery', 'NotTalking2U', 'PodBayDoor', 'PodBayDoorResponse', 'RealNameQuery', 'SelfAware', 'Shutup', 'Swearing', 'Thanks', 'TimeQuery', 'UnderstandQuery', 'WhoAmI']
118 unique lemmatized words ["'s", 'a', 'about', 'adam', 'adios', 'am', 'any', 'anyone', 'are', 'aware', 'bay', 'be', 'bella', 'bored', 'by', 'bye', 'call', 'camera', 'can', 'cheering', 'clever', 'communicating', 'comprendo', 'conscious', 'could', 'do', 'doing', 'door', 'enough', 'for', 'friend', 'fuck', 'genious', 'get', 'girl', 'give', 'good', 'goodbye', 'gossip', 'got', 'great', 'have', 'hear', 'hello', 'help', 'helpful', 'hi', 'hola', 'hope', 'how', 'hya', 'i', 'identify', 'in', 'intelligent', 'is', 'it', 'joke', 'know', 'later', 'laugh', 'make', 'me', 'mean', 'meant', 'more', 'my', "n't", 'name', 'need', 'not', 'off', 'ok', 'open', 

In [50]:
# create the training data
training = []
# create empty array for the output
output_empty = [0] * len(classes)
# training set, bag of words for every sentence
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    word_patterns = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
        
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle the features and make numpy array
random.shuffle(training)
training = np.array(training)
# create training and testing lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data is created")

Training data is created


C:\Users\User\AppData\Local\Temp/ipykernel_4236/3348361106.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [51]:
# deep neural networds model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
# opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#Training and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model is created")

Epoch 1/200
29/29 [==============================] - 0s 589us/step - loss: 3.0693 - accuracy: 0.0690
Epoch 2/200
29/29 [==============================] - 0s 589us/step - loss: 2.9911 - accuracy: 0.1172
Epoch 3/200
29/29 [==============================] - 0s 589us/step - loss: 2.8510 - accuracy: 0.1793
Epoch 4/200
29/29 [==============================] - 0s 554us/step - loss: 2.7300 - accuracy: 0.2690
Epoch 5/200
29/29 [==============================] - 0s 572us/step - loss: 2.4712 - accuracy: 0.3172
Epoch 6/200
29/29 [==============================] - 0s 571us/step - loss: 2.2686 - accuracy: 0.3793
Epoch 7/200
29/29 [==============================] - 0s 571us/step - loss: 2.1642 - accuracy: 0.3862
Epoch 8/200
29/29 [==============================] - 0s 554us/step - loss: 1.8877 - accuracy: 0.4690
Epoch 9/200
29/29 [==============================] - 0s 571us/step - loss: 1.7096 - accuracy: 0.5379
Epoch 10/200
29/29 [==============================] - 0s 571us/step - loss: 1.6482 - accura

29/29 [==============================] - 0s 554us/step - loss: 0.1350 - accuracy: 0.9655
Epoch 82/200
29/29 [==============================] - 0s 554us/step - loss: 0.1302 - accuracy: 0.9724
Epoch 83/200
29/29 [==============================] - 0s 554us/step - loss: 0.0862 - accuracy: 0.9793
Epoch 84/200
29/29 [==============================] - 0s 571us/step - loss: 0.1045 - accuracy: 0.9862
Epoch 85/200
29/29 [==============================] - 0s 572us/step - loss: 0.0996 - accuracy: 0.9655
Epoch 86/200
29/29 [==============================] - 0s 589us/step - loss: 0.1577 - accuracy: 0.9379
Epoch 87/200
29/29 [==============================] - 0s 571us/step - loss: 0.0948 - accuracy: 0.9793
Epoch 88/200
29/29 [==============================] - 0s 571us/step - loss: 0.0859 - accuracy: 0.9793
Epoch 89/200
29/29 [==============================] - 0s 554us/step - loss: 0.0900 - accuracy: 0.9724
Epoch 90/200
29/29 [==============================] - 0s 571us/step - loss: 0.0962 - accuracy: 

29/29 [==============================] - 0s 607us/step - loss: 0.0454 - accuracy: 0.9862
Epoch 161/200
29/29 [==============================] - 0s 554us/step - loss: 0.0356 - accuracy: 1.0000
Epoch 162/200
29/29 [==============================] - 0s 571us/step - loss: 0.0462 - accuracy: 0.9931
Epoch 163/200
29/29 [==============================] - 0s 571us/step - loss: 0.0657 - accuracy: 0.9793
Epoch 164/200
29/29 [==============================] - 0s 584us/step - loss: 0.0755 - accuracy: 0.9862
Epoch 165/200
29/29 [==============================] - 0s 553us/step - loss: 0.0737 - accuracy: 0.9724
Epoch 166/200
29/29 [==============================] - 0s 554us/step - loss: 0.0471 - accuracy: 0.9862
Epoch 167/200
29/29 [==============================] - 0s 625us/step - loss: 0.0588 - accuracy: 0.9793
Epoch 168/200
29/29 [==============================] - 0s 571us/step - loss: 0.0903 - accuracy: 0.9655
Epoch 169/200
29/29 [==============================] - 0s 589us/step - loss: 0.0870 - a

In [52]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('Intent.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for words that exist in sentence

def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))

def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['intent']== tag):
            result = random.choice(i['responses'])
            break
    return result

#Creating tkinter GUI
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatBox.config(state=NORMAL)
        ChatBox.insert(END, "You: " + msg + '\n\n')
        ChatBox.config(foreground="#446665", font=("Verdana", 12 )) 

        ints = predict_class(msg)
        res = getResponse(ints, intents)

        ChatBox.insert(END, "Bot: " + res + '\n\n')           

        ChatBox.config(state=DISABLED)
        ChatBox.yview(END)

root = Tk()
root.title("Chatbot")
root.geometry("800x500")
root.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatBox = Text(root, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatBox.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
ChatBox['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#f9a602", activebackground="#3c9d9b",fg='#000000',
                    command= send )

#Create the box to enter message
EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)

#Place all components on the screen
scrollbar.place(x=776,y=6, height=386)
ChatBox.place(x=6,y=6, height=386, width=770)
EntryBox.place(x=128, y=401, height=90, width=765)
SendButton.place(x=6, y=401, height=90)

root.mainloop()